In [1]:
import os
import json

import session_processing_helper as helper
import utils

import pandas as pd

In [2]:
data_dir = '/Users/rebekahzhang/data/behavior_data'
exp = "exp2"
data_folder = os.path.join(data_dir, exp)
print(data_folder)

/Users/rebekahzhang/data/behavior_data/exp2


In [7]:
with open('exp_cohort_info.json', 'r') as f:
    training_info = json.load(f)

In [15]:
cohort_info = training_info['cohorts']

In [16]:
def add_cohort_column(sessions_all, cohort_info):
    """Add cohort column based on mouse name and cohort info."""
    # Create reverse mapping
    mouse_to_cohort = {}
    for cohort, mice in cohort_info.items():
        for mouse in mice:
            mouse_to_cohort[mouse] = cohort
    
    # Add cohort column
    sessions_all['cohort'] = sessions_all['mouse'].map(mouse_to_cohort)
    
    # Check for any unmapped mice
    unmapped = sessions_all[sessions_all['cohort'].isna()]
    if not unmapped.empty:
        print(f"Warning: {len(unmapped)} mice not found in cohort info:")
        print(unmapped['mouse'].unique())
    
    return sessions_all

In [17]:
_, sessions_training = helper.generate_session_logs(data_folder, save_logs=False)
session_info_list = []
for _, session_info in sessions_training.iterrows():
    events_processed = pd.read_csv(utils.generate_events_processed_path(data_folder, session_info), low_memory=False)
    session_basics = helper.get_session_basics(events_processed)
    session_basics['dir'] = session_info['dir']
    session_info_list.append(session_basics)
sessions_info = pd.DataFrame(session_info_list)
corrected_sessions_training = pd.merge(sessions_training, sessions_info, on="dir")
corrected_sessions_training = corrected_sessions_training.drop(columns=['index', 'total_reward', 'total_trial', 'total_reward'])
corrected_sessions_training = corrected_sessions_training.groupby('mouse', group_keys=False).apply(helper.assign_session_numbers)
corrected_sessions_training = add_cohort_column(corrected_sessions_training, cohort_info)
# if save_log:
utils.save_as_csv(df=corrected_sessions_training, folder=data_folder, filename=f'sessions_training_{exp}.csv')

1831 sessions in total
